In [2]:
from werpy_weighted import werpy
import whisper
from whisper.tokenizer import get_tokenizer

In [3]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [4]:
import os, sys, glob, shutil
import argparse
import json
import yaml
import numpy as np
from pprint import pprint
import logging
import os
import os.path as op

import torch
import torch.nn.functional as F
import numpy as np
import torchaudio
import pandas as pd
from tqdm.notebook import tqdm

In [7]:
tokenizer = get_tokenizer(multilingual=False)  # use multilingual=True if using multilingual model
number_tokens = [
    i 
    for i in range(tokenizer.eot)
    if all(c in "0123456789" for c in tokenizer.decode([i]).removeprefix(" "))
]

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

whisper_model = whisper.load_model('base.en').to(device)

In [9]:
split_txt_val_path = r'.\preprocessed_data\LibriTTS\val_ming.txt'
val_uids = []
gt_transcripts_dict = {}
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])
        gt_transcripts_dict[line.split('|')[0]] = line.split('|')[-1]

In [10]:
syn_folder = r'.\output\0714lb\result\LibriTTS\wav\synthesized'

In [35]:
%%time
uids = []
whisper_transcripts = []
gt_transcripts = []

for uid in tqdm(val_uids):
    uids.append(uid)
    output = whisper_model.transcribe(
            op.join(syn_folder, f"{uid}.wav"),
            suppress_tokens=[-1] + number_tokens,
    )
    whisper_transcript = output["text"]
    gt_transcript = gt_transcripts_dict[uid]

    whisper_transcripts.append(whisper_transcript)
    gt_transcripts.append(gt_transcript)


  0%|          | 0/512 [00:00<?, ?it/s]

CPU times: total: 11min 40s
Wall time: 4min 43s


In [36]:
whisper_df_0703a_1M8 = pd.DataFrame({
    'uid': uids,
    'whisper_transcripts': whisper_transcripts,
    'gt_transcripts': gt_transcripts,
})

In [37]:
whisper_df_0703a_1M8.head()

,uid,whisper_transcripts,gt_transcripts
0,5400_3587_000037_000001,Even the cunning blandishments of a very attr...,even the cunning blandishments of a very attra...
1,8772_270777_000011_000003,The child had tried again and again to see it...,the child had tried again and again to see it ...
2,596_139612_000023_000003,"Now, please, give me the ashes of the mortar ...","now please give me the ashes of the mortar, as..."
3,4899_32639_000029_000000,"Oh, your seat has changed, said she.","oh, your seat is changed,"" said she.\n"
4,5304_109507_000023_000000,"It's three minutes of twelve, he said.","""it's three minutes of twelve,"" he said.\n"


In [38]:
whisper_df_0703a_1M8.to_csv(r".\jupyter_walkthrough\metrics\whisper_df_0703a_1M8_syn.csv", index=False)

In [39]:
whisper_df_0703a_1M8 = pd.read_csv(r".\jupyter_walkthrough\metrics\whisper_df_0703a_1M8_syn.csv")

In [40]:
normalized_ref = werpy.normalize(
    whisper_df_0703a_1M8['gt_transcripts'].values.tolist(),
)
normalized_hypo = werpy.normalize(
    whisper_df_0703a_1M8['whisper_transcripts'].values.tolist(),
)

In [41]:
normalized_ref[0]

'even the cunning blandishments of a very attractive monkey that always had entertained the children on their numerous visits failed to interest her now mamsie would be worrying she knew and besides the sight of so many birds eating their suppers out of generously full seed cups only filled her heart with remorse as she thought of poor cherry and his empty one'

In [42]:
normalized_hypo[0]

'even the cunning blandishments of a very attractive monkey that always had entertained the children on their numerous visits failed to interest her now mamsey would be worrying she knew and besides the sight of so many birds eating their suppers out of generously full seatcaps only filled her heart with the remorse as she thought it poor cherry and his empty one'

In [43]:
%%time
summary = werpy.summary(normalized_ref, normalized_hypo)
summary

CPU times: total: 188 ms
Wall time: 209 ms


,wer,ld,m,insertions,deletions,substitutions,inserted_words,deleted_words,substituted_words
0,0.285714,18,63,1,1,3,[the],[seed],"[(mamsie, mamsey), (cups, seatcaps), (of, it)]"
1,0.000000,0,19,0,0,0,[],[],[]
2,0.000000,0,20,0,0,0,[],[],[]
3,0.571429,4,7,0,0,1,[],[],"[(is, has)]"
4,0.000000,0,7,0,0,0,[],[],[]
...,...,...,...,...,...,...,...,...,...
507,0.000000,0,7,0,0,0,[],[],[]
508,0.875000,7,8,1,0,1,[taylor],[],"[(taylori, i)]"
509,0.000000,0,24,0,0,0,[],[],[]
510,0.000000,0,4,0,0,0,[],[],[]


In [44]:
I = summary['insertions'].sum()
D = summary['deletions'].sum()
S = summary['substitutions'].sum()
M = summary['m'].sum()

In [45]:
summary.to_csv(r".\jupyter_walkthrough\metrics\0703a_1M8_wer_summary.csv", index=False)

In [46]:
wer = (I + D + S) / M

In [47]:
wer

0.06090036563071298

In [48]:
I + D + S

533

In [49]:
M

8752

In [52]:
I, D, S, M

(70, 137, 326, 8752)

In [51]:
I/M, D/M, S/M

(0.007998171846435101, 0.015653564899451553, 0.03724862888482632)